# Goal

The goal of this notebook is to demonstrate the usage of the new, multi-objective, and interactive version of the Tomograph.

## Goals for the Tomograph2

1. Move away from matplotlib to either plotly, or bokeh. 
2. Make sure that:
    1. All scales are the same so that they are comparable.
    2. We should be able to plot not just predictions but also individual observations as a scatterplot. 
    3. We should be able to link the points across multiple charts to investigate them more deeply (bokeh seems to be the winner here)
    4. We should be able to connect multiple tomographs to the same optimizer, as any given analysis might focus on different aspects and each aspect should have it's own graphs.
3. The Tomograph produces a set of heatmaps that belong to 2D cross-sections of the higher-dimensional hypercube. All these cross-sections share a single point (often the optimum). The user should be able to:
    1. Select one of the predefined points (different types of optima, maybe optima for different contexts)
    2. View the resulting cross sections and sensitivity analysis for such a point
    3. Be able to use sliders to adjust the point so that they can easily "walk the hypercube"
    

Plotly has the advantage of also enabling 3D visualizations of the pareto frontier and generates surface plots etc. But bokeh has built-in support for linking data. 

Given what we need, we will start with bokeh, for dabl-style and tomograph plots, and then switch to plotly for parameter interactions, and 3D pareto visualizations.

https://docs.bokeh.org/en/latest/index.html

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Create and train the Optimizer

In [2]:
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory, objective_function_config_store

#objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_2_mutually_exclusive_polynomials")
objective_function_config = objective_function_config_store.get_config_by_name("three_level_quadratic")
#objective_function_config = objective_function_config_store.get_config_by_name("2d_hypersphere_minimize_none")
#objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_waves_3_params_2_objectives_half_pi_phase_difference")

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)
optimization_problem = objective_function.default_optimization_problem

In [3]:
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
optimizer_factory = BayesianOptimizerFactory()

if len(optimization_problem.objectives) == 1:
    optimizer_config = bayesian_optimizer_config_store.default
else:
    optimizer_config = bayesian_optimizer_config_store.get_config_by_name("default_with_random_near_incumbent_config")
    
optimizer = optimizer_factory.create_local_optimizer(
    optimizer_config=optimizer_config,
    optimization_problem=optimization_problem
)

03/18/2021 21:47:41 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  40 -    create_local_optimizer() ] Creating a bayesian optimizer with config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.7,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_m

In [4]:
params_df = objective_function.parameter_space.random_dataframe(num_samples=100)
objectives_df = objective_function.evaluate_dataframe(params_df)
optimizer.register(parameter_values_pandas_frame=params_df, target_values_pandas_frame=objectives_df)

03/18/2021 21:47:41 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 100 parameters and 100 objectives.


# Plot observations using ModelTomograph2

In [5]:
from bokeh.io import output_notebook, show
from mlos.OptimizerMonitoring.Tomograph.ModelTomograph2 import ModelTomograph2    
    
output_notebook()
tomograph = ModelTomograph2(optimizer=optimizer)
report = tomograph.get_report()
show(report)

Loading BokehJS ...

In [6]:
# Let's run through a bunch of iterations.
#
for _ in range(100):
    suggestion_df = optimizer.suggest().to_dataframe()
    objectives_df = objective_function.evaluate_dataframe(suggestion_df)
    optimizer.register(suggestion_df, objectives_df)
    

03/18/2021 21:47:43 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   suggest() ] Producing random suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": 6.254506370561046,
  "low_quadratic_params.x_2": 13.349050125848265
}
03/18/2021 21:47:43 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:43 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 189 -                   suggest() ] Produced a guided suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": -29.808631909863976,
  "low_quadratic_params.x_2": -69.48166671027602
}
03/18/2021 21:47:43 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:43 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   sugges

03/18/2021 21:47:47 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:47 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   suggest() ] Producing random suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": 79.64305107123158,
  "low_quadratic_params.x_2": 54.66539792939321
}
03/18/2021 21:47:47 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:47 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 189 -                   suggest() ] Produced a guided suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": -20.10283067211121,
  "low_quadratic_params.x_2": -70.83187866772305
}
03/18/2021 21:47:47 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register()

03/18/2021 21:47:49 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:50 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   suggest() ] Producing random suggestion: {
  "vertex_height": 5,
  "medium_quadratic_params.x_1": 55.6193272945105,
  "medium_quadratic_params.x_2": 26.45004789130465
}
03/18/2021 21:47:50 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:50 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 189 -                   suggest() ] Produced a guided suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": 5.736815135102006,
  "low_quadratic_params.x_2": -65.29472224705772
}
03/18/2021 21:47:50 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register()

03/18/2021 21:47:53 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:53 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 189 -                   suggest() ] Produced a guided suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": -47.19227150303005,
  "low_quadratic_params.x_2": -71.20397296743769
}
03/18/2021 21:47:53 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:53 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   suggest() ] Producing random suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": 1.6948782532993931,
  "low_quadratic_params.x_2": 90.83193275081717
}
03/18/2021 21:47:53 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register(

03/18/2021 21:47:56 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:56 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   suggest() ] Producing random suggestion: {
  "vertex_height": 5,
  "medium_quadratic_params.x_1": -42.72711153360718,
  "medium_quadratic_params.x_2": -48.920599661098876
}
03/18/2021 21:47:56 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:57 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 189 -                   suggest() ] Produced a guided suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": -14.628832535481394,
  "low_quadratic_params.x_2": -72.29390588954034
}
03/18/2021 21:47:57 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  regi

03/18/2021 21:47:59 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:59 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 189 -                   suggest() ] Produced a guided suggestion: {
  "vertex_height": "low",
  "low_quadratic_params.x_1": -42.50821926752246,
  "low_quadratic_params.x_2": -66.94699299534268
}
03/18/2021 21:47:59 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 1 parameters and 1 objectives.
03/18/2021 21:47:59 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 185 -                   suggest() ] Producing random suggestion: {
  "vertex_height": 5,
  "medium_quadratic_params.x_1": 30.27173837211481,
  "medium_quadratic_params.x_2": 44.32568797740649
}
03/18/2021 21:47:59 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register

In [7]:
show(tomograph.get_report())